In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from cmdstanpy import CmdStanModel
from baynes.plotter import FitPlotter
from baynes.model_utils import get_stan_file, get_compiler_kwargs
from baynes.probability import hdi
import pandas as pd
from scipy import stats
import arviz as az
sns.set_style('ticks')
#sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=1.6)
plt.rc("axes.spines", top=False, right=False)

In [ ]:
def cov_exp_quad(x1, x2, alpha, rho):
    x1 = np.asarray(x1)
    x2 = np.asarray(x1)
    k = []
    for x in x1:
        k.append((x-x2)**2)
    return alpha**2 * np.exp(-np.array(k)/(2*rho**2))

def cov_periodic(x1, x2, alpha, rho, p):
    x1 = np.asarray(x1)
    x2 = np.asarray(x1)
    k = []
    for x in x1:
        k.append(np.abs(x-x2))
    return alpha**2 * np.exp(-2*np.sin(np.pi*np.array(k)/p)**2/(rho**2))


fplot = FitPlotter(fig_ratio=1, output_dir="/home/pietro/work/TESI/thesis/figures/ch2/GP/", output_format='.pdf')

In [ ]:
N2=101
x = np.linspace(0,10, N2)
N1=7
x1 = np.sort(np.random.choice(x, N1))
y1 = 0.3*x1 + np.random.normal(0,0.1,N1)+ 0.5* np.sin(x1)
plt.scatter(x1, y1)

In [ ]:
stan_file = get_stan_file('GP_quad.stan')
print(stan_file)
model = CmdStanModel(stan_file=stan_file, **get_compiler_kwargs())

In [ ]:
get_compiler_kwargs()

In [ ]:
data={'N1':N1, 'N2':N2, 'x1':x1, 'x2':x, 'y1':y1}
#data={'N':N1, 'x':x1, 'y':y1, 'prior':0}

fit = model.sample(data,
                   chains=4,
                   iter_warmup=500,
                   iter_sampling=500,
                   save_warmup=True,
                   show_progress=True,
                   adapt_delta=0.9,
                   inits=1)
print(fit.diagnose())
fplot.add_fit(fit)
df = fit.draws_pd(['y2']).transpose()
df['x']=x
df = df.melt(id_vars='x')
ax = sns.lineplot(df, x='x', y='value', errorbar=hdi)
sns.scatterplot(x=x1, y=y1, ax=ax)

In [ ]:
np.mean(fit.draws_pd(['alpha']).to_numpy())

In [ ]:
means = fit.draws_pd(['alpha', 'rho']).mean(axis=0)
z=cov_exp_quad(x, x, means['alpha'], means['rho'])
plt.pcolor(x,x,z)


In [ ]:
stan_file = get_stan_file('GP_periodic.stan')
modelper = CmdStanModel(stan_file=stan_file, **get_compiler_kwargs())

In [ ]:
data={'N1':N1, 'N2':N2, 'x1':x1, 'x2':x, 'y1':y1}
#data={'N':N1, 'x':x1, 'y':y1, 'prior':0}

fit = modelper.sample(data,
                   chains=4,
                   iter_warmup=500,
                   iter_sampling=500,
                   save_warmup=True,
                   show_progress=True)
print(fit.diagnose())
fplot.add_fit(fit)
df = fit.draws_pd(['y2']).transpose()
df['x']=x
df = df.melt(id_vars='x')
ax = sns.lineplot(df, x='x', y='value', errorbar=hdi)
sns.scatterplot(x=x1, y=y1, ax=ax)

In [ ]:
z=cov_periodic(x, x, np.mean(fit.draws_pd(['alpha']).to_numpy()), 
np.mean(fit.draws_pd(['rho']).to_numpy()),
np.mean(fit.draws_pd(['p']).to_numpy()))
plt.pcolor(x,x,z)

In [ ]:
def cov_exp_quad(x1, x2, alpha, rho):
    x1 = np.asarray(x1)
    x2 = np.asarray(x1)
    k = []
    for x in x1:
        k.append((x-x2)**2)
    return alpha**2 * np.exp(-np.array(k)/(2*rho**2))

def cov_periodic(x1, x2, alpha, rho, p):
    x1 = np.asarray(x1)
    x2 = np.asarray(x1)
    k = []
    for x in x1:
        k.append(np.abs(x-x2))
    return alpha**2 * np.exp(-2*np.sin(np.pi*np.array(k)/p)**2/(rho**2))

In [ ]:
x=np.linspace(0,10,201)
z=cov_exp_quad(x, x, 1, 1)
zp = cov_periodic(x,x,1,1,3)

In [ ]:
plt.set_cmap('rainbow')
plt.pcolor(x,x,z)

In [ ]:
plt.pcolor(x,x,zp)